## SynNet (vocab & word matrix)

In [1]:
from poems_model import PoemsModel

poems_data = './data/poems_model.pickle'
w2v_file = 'c:/data/ruscorpora_1_300_10.bin.gz'
pm = PoemsModel(poems_data, w2v_file)


C:\Users\Drafter\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Slow version of gensim.models.doc2vec is being used
2017-05-29 15:05:55,172 : INFO : loading projection weights from c:/data/ruscorpora_1_300_10.bin.gz


loading w2v_model...


2017-05-29 15:06:13,334 : INFO : loaded (161301, 300) matrix from c:/data/ruscorpora_1_300_10.bin.gz


word2vec model 'c:/data/ruscorpora_1_300_10.bin.gz' loaded
reading pickle poems model...
vectorizing model...
model is loaded


In [208]:
from gensim.models.keyedvectors import KeyedVectors

def synNet(w2v: KeyedVectors, word: str, breadth=10, depth=3) -> dict:
    net = {}
    net[0] = {word}
    vocab = set()
    vocab.add(word)
    
    def get_associations(word: str, level: int):
        nonlocal net, vocab
        similars = [a[0] for a in w2v.most_similar(word, topn=breadth) if a[0] not in vocab]
        net[level] = net.get(level, {}).union(similars)
        vocab = vocab.union(similars)
        # print(similars, vocab)
        if level < depth:
            for sim in similars:
                get_associations(sim, level + 1)
    
    def gather_net(word: str, level: int):
        nonlocal net
        similars = [a[0] for a in w2v.most_similar(word, topn=breadth)]
        # net[level] = net.get(level, set()).union(similars)
        if level not in net:
            net[level] = set()
        if level < depth:
            for sim in similars:
                if sim not in net[level-1] | net[level]:
                    gather_net(sim, level + 1)
        net[level] = net[level].union(similars)

    def reduce_net():
        nonlocal net
        for i in range(depth):
            i_set = net[i]
            for j in range(i+1, depth+1):
                j_set = net[j]
                for w in i_set:
                    if w in j_set:
                        j_set.remove(w)
                
    # get_associations(word, 1)
    gather_net(word, 1)
    reduce_net()
    return net
            
    

In [209]:
%%time
synNet(pm.w2v, 'свеча_NOUN')

Wall time: 1.24 s


{0: {'свеча_NOUN'},
 1: {'восковой_ADJ',
  'зажигать_VERB',
  'засвечать_VERB',
  'кенкета_NOUN',
  'лампада_NOUN',
  'лампадка_NOUN',
  'огарок_NOUN',
  'подсвечник_NOUN',
  'свечка_NOUN',
  'шандал_NOUN'},
 2: {'абажурчик_NOUN',
  'воск_NOUN',
  'восково_NOUN',
  'гореть_VERB',
  'догорать_VERB',
  'жирандоль_NOUN',
  'зажженый_ADJ',
  'зажигаться_VERB',
  'затепливать_VERB',
  'каганец_NOUN',
  'канделябр_NOUN',
  'карсель_NOUN',
  'кенкет_NOUN',
  'киот_NOUN',
  'коптилка_NOUN',
  'лампадный_ADJ',
  'лампадочка_NOUN',
  'лампешка_NOUN',
  'люстра_NOUN',
  'нагорать_VERB',
  'нагорелый_ADJ',
  'нащепать_VERB',
  'ночник_NOUN',
  'ночничок_NOUN',
  'огарочек_NOUN',
  'ослопной_ADJ',
  'погасать_VERB',
  'подставка_NOUN',
  'полузавешивать_VERB',
  'полузадернуть_VERB',
  'потушить_VERB',
  'припечек_NOUN',
  'простеночный_ADJ',
  'светильня_NOUN',
  'свечечка_NOUN',
  'свечница_NOUN',
  'свещник_NOUN',
  'смолье_NOUN',
  'теплиться_VERB',
  'тушить_VERB'},
 3: {'абажур_NOUN',
  'архи

In [108]:
pm.w2v.similarity('сон_NOUN', 'разбудить_VERB')

0.46540949340518623

In [105]:
pm.w2v.similarity('сон_NOUN', 'дремать_VERB')

0.41081592150334684

In [22]:
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

def semantic_levels(w2v: KeyedVectors, base_word: str, portions=[0.2, 0.3, 0.5], vocab_count=100) -> (dict, np.ndarray):
    if base_word not in w2v.vocab:
        return {}, []
    
    levels = {}
    levels[0] = {base_word}
    for level in range(1, len(portions) + 1):
        levels[level] = set()
    matrix = [w2v.word_vec(base_word)]    
            
    similars =  w2v.most_similar(base_word, topn=vocab_count)
    
    def gather_by_similarity():
        max_sim = similars[0][1]
        sim_notches = [max_sim * (1 - portion) for portion in portions]
        for word, sim in similars:
            for level, notch in enumerate(sim_notches):
                if sim >= notch:
                    levels[level+1].add(word)
                    matrix.append(w2v.word_vec(word) * (1 - portions[level]))
                    break
    
    def gather_by_count():
        notch = 0
        next_notch = 0
        for level, portion in enumerate(portions):
            next_notch = notch + int(vocab_count * portion)
            for i in range(notch, next_notch):
                word = similars[i][0]
                levels[level+1].add(word)
                matrix.append(w2v.word_vec(word) * (1 - portions[level]))
            notch = next_notch
    
    gather_by_count()
    return levels, np.vstack(matrix)

In [24]:
%%time
semantic_levels(pm.w2v, 'пилорама_NOUN', vocab_count=50)

Wall time: 39 ms


({0: {'пилорама_NOUN'},
  1: {'коксохим_NOUN',
   'кормокухня_NOUN',
   'лесопилка_NOUN',
   'лесопильня_NOUN',
   'лесоучасток_NOUN',
   'пила_NOUN',
   'пилиться_VERB',
   'скреперный_ADJ',
   'трелевщик_NOUN',
   'узкоколейка_NOUN'},
  2: {'бульдозер_NOUN',
   'вагончик_NOUN',
   'вторчермет_NOUN',
   'лежневка_NOUN',
   'лесопильный_ADJ',
   'пилить_VERB',
   'пильня_NOUN',
   'сезонница_NOUN',
   'столярка_NOUN',
   'стояночный_ADJ',
   'стройка_NOUN',
   'трактор_NOUN',
   'циркулярка_NOUN',
   'электродойка_NOUN',
   'электропила_NOUN'},
  3: {'автопоилка_NOUN',
   'бензопила_NOUN',
   'бревно_NOUN',
   'гараж_NOUN',
   'грабарка_NOUN',
   'запань_NOUN',
   'камнедробилка_NOUN',
   'колюха_NOUN',
   'комбинатский_ADJ',
   'лесозавод_NOUN',
   'литейка_NOUN',
   'молоток_NOUN',
   'ножовка_NOUN',
   'овчарник_NOUN',
   'овчарный_ADJ',
   'ремонтно-механический_ADJ',
   'свинарник_NOUN',
   'свинарня_NOUN',
   'соломорезка_NOUN',
   'строгальный_ADJ',
   'телятник_NOUN',
   'уклад

In [ ]:
pm.w2v.word_vec('стакан_NOUN')

## Семантические запросы

In [35]:
_, query_mx = pm.semantic_levels('автобус_NOUN', vocab_count=500)

In [36]:
%%time
pm.similar_poems(query_mx, topn=10)

Wall time: 1.37 s


[('мой домик из рекламных стендов\nстоит у старого шоссе\nи редкий грузовик проехав\nмигает фарами в окноочень хороший\n',
  0.16981978914270282),
 ('ты села в желтую машину\nи унеслась за горизонт\nи у тебя сегодня будет\nроман с водителем такси\n',
  0.15520665196497224),
 ('маршрутка обгоняя транспорт\nнесется к станции метро\nв ней сто семнадцать пассажиров\nсрослись в единый организм\n',
  0.1531956979565208),
 ('спортивный самолёт дождался\nкоманд диспетчера на старт\nвключил мотор надел кроссовки\nи плавно начал свой разбег\n',
  0.1524336337091442),
 ('трамвай на рельсах поскользнулся\nстальными дисками колес\nи покатился по проспекту\nнаматывая провода\n',
  0.14889686037182998),
 ('мне снилась осень дождь и слякоть\nзонты плывущие в туман\nи на трамвайной остановке\nдевчонка в клетчатом пальто\n',
  0.14828851415914984),
 ('шаинский едет на квартирник\nи выкатив из гаража\nсвой старенький стейнвей мурлычет\nпод нос про голубой вагон\n',
  0.14511992859983158),
 ('в вонючей ст

In [37]:
%%time
pm.similar_poems('автобус', topn=10)

Wall time: 201 ms


[('маршрутка обгоняя транспорт\nнесется к станции метро\nв ней сто семнадцать пассажиров\nсрослись в единый организм\n',
  0.36026285170569239),
 ('идет учительница наша\nи с ней идет такая мощ\nчто расступаются трамваи\nи глохнут поезда метро\n',
  0.34719832330275019),
 ('ты села в желтую машину\nи унеслась за горизонт\nи у тебя сегодня будет\nроман с водителем такси\n',
  0.34566201314811734),
 ('мой домик из рекламных стендов\nстоит у старого шоссе\nи редкий грузовик проехав\nмигает фарами в окноочень хороший\n',
  0.33587739991481502),
 ('на текстилях открылись двери\nвагонов поезда метро\nи надписи не прислоняться\nостались в воздухе висеть\n',
  0.32272098657089782),
 ('мне снилась осень дождь и слякоть\nзонты плывущие в туман\nи на трамвайной остановке\nдевчонка в клетчатом пальто\n',
  0.31977505489497698),
 ('спортивный самолёт дождался\nкоманд диспетчера на старт\nвключил мотор надел кроссовки\nи плавно начал свой разбег\n',
  0.30832762718200685),
 ('я закольцовываю счастье